# Rasa TED Demo

--------------

In [4]:
import os

--------------

In [9]:
from rasa.nlu.training_data import load_data 
from rasa.nlu.model import Trainer
from rasa.nlu import config
from rasa.nlu.model import Interpreter

In [2]:
!ls ../../rasa-ted-demo

__init__.py  config.yml       data	  endpoints.yml  requirements.txt
actions.py   credentials.yml  domain.yml  readme.md


In [5]:
root_dir = '../../rasa-ted-demo'

In [7]:
data_path = os.path.abspath(os.path.join(root_dir, 'data/nlu.md'))
config_path = os.path.abspath(os.path.join(root_dir, 'config.yml'))
model_dir = os.path.abspath(os.path.join(root_dir, 'models'))

----------------

In [10]:
training_data = load_data(data_path)

In [11]:
trainer = Trainer(config.load(config_path))

In [12]:
trainer.train(training_data)

/usr/local/lib/python3.6/dist-packages/rasa/utils/common.py:363: UserWarning: You specified 'DIET' to train entities, but no entities are present in the training data. Skip training of entities.
Epochs:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/rasa/utils/tensorflow/model_data.py:386: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  final_data[k].append(np.concatenate(np.array(v)))
Epochs: 100%|██████████| 100/100 [00:10<00:00,  9.56it/s, t_loss=1.269, i_loss=0.133, i_acc=1.000]


In [13]:
model_directory = trainer.persist(model_dir, fixed_model_name="current")

----------------

In [14]:
interpreter = Interpreter.load(os.path.join(model_dir, 'current'))

In [15]:
text = 'count'
interpreter.parse(text)

{'intent': {'name': 'start_count', 'confidence': 0.9999994039535522},
 'entities': [],
 'intent_ranking': [{'name': 'start_count', 'confidence': 0.9999994039535522},
  {'name': 'bot_challenge', 'confidence': 4.4161035361867107e-07},
  {'name': 'goodbye', 'confidence': 7.065003870820874e-08},
  {'name': 'greet', 'confidence': 3.9161669462828286e-08},
  {'name': 'confirm', 'confidence': 2.158538237040375e-08}],
 'response_selector': {'default': {'response': {'name': None,
    'confidence': 0.0},
   'ranking': [],
   'full_retrieval_intent': None}},
 'text': 'count'}

------------------

In [16]:
import asyncio

In [17]:
from rasa.core import config
from rasa.core.agent import Agent

In [21]:
!ls ../../rasa-ted-demo/*

../../rasa-ted-demo/__init__.py      ../../rasa-ted-demo/domain.yml
../../rasa-ted-demo/actions.py	     ../../rasa-ted-demo/endpoints.yml
../../rasa-ted-demo/config.yml	     ../../rasa-ted-demo/readme.md
../../rasa-ted-demo/credentials.yml  ../../rasa-ted-demo/requirements.txt

../../rasa-ted-demo/data:
nlu.md	stories.md

../../rasa-ted-demo/models:
current


In [18]:
root_dir = '../../rasa-ted-demo'

In [22]:
domain_config = os.path.abspath(os.path.join(root_dir, 'domain.yml'))
policy_config = os.path.abspath(os.path.join(root_dir, 'config.yml'))
training_data_file = os.path.abspath(os.path.join(root_dir, 'data/stories.md'))
model_dir = os.path.abspath(os.path.join(root_dir, 'models'))
policy_fixed_model_name="dialogue"

------------

In [23]:
policies = config.load(policy_config)

/usr/local/lib/python3.6/dist-packages/rasa/core/policies/ensemble.py:310: FutureWarning: 'KerasPolicy' is deprecated and will be removed in version 2.0. Use 'TEDPolicy' instead.
  policy_object = constr_func(**policy)


In [24]:
agent = Agent(domain_config,
              policies=policies)

In [25]:
training_data = await agent.load_data(training_data_file)

Processed Story Blocks: 100%|██████████| 7/7 [00:00<00:00, 63.73it/s, # trackers=17]


In [26]:
agent.train(training_data)

Processed trackers: 100%|██████████| 7/7 [00:00<00:00, 18.52it/s, # actions=85]
Processed actions: 85it [00:00, 438.48it/s, # examples=85]
Processed trackers: 100%|██████████| 265/265 [00:23<00:00, 11.11it/s, # actions=141]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 28)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7808      
_________________________________________________________________
dense (Dense)                (None, 23)                759       
_________________________________________________________________
activation (Activation)      (None, 23)                0         
Total params: 8,567
Trainable params: 8,567
Non-trainable params: 0
_________________________________________________________________
Train on 141 samples
Epoch 1/200
141/141 [==============================] - 4s 26ms/sample - loss: 3.0733 - accuracy: 0.1489
Epoch 2/200
141/141 [==============================] - 0s 348us/sample - loss: 2.9800 - accuracy: 0.4113
Epoch 3/200
141/141 [=

In [27]:
agent_model_path = os.path.join(model_dir, policy_fixed_model_name)

In [28]:
agent.persist(agent_model_path)

In [29]:
!ls ../../rasa-ted-demo/*

../../rasa-ted-demo/__init__.py      ../../rasa-ted-demo/domain.yml
../../rasa-ted-demo/actions.py	     ../../rasa-ted-demo/endpoints.yml
../../rasa-ted-demo/config.yml	     ../../rasa-ted-demo/readme.md
../../rasa-ted-demo/credentials.yml  ../../rasa-ted-demo/requirements.txt

../../rasa-ted-demo/data:
nlu.md	stories.md

../../rasa-ted-demo/models:
current  dialogue


In [36]:
!ls ../../rasa-ted-demo/models/dialogue/core

domain.json  metadata.json		 policy_1_KerasPolicy
domain.yml   policy_0_MemoizationPolicy  policy_2_MappingPolicy


---------------------

In [37]:
from rasa.core.agent import Agent
from rasa.core.interpreter import NaturalLanguageInterpreter
from rasa.core.utils import EndpointConfig

In [38]:
model_directory = os.path.join(model_dir, "current")

In [39]:
interpreter = NaturalLanguageInterpreter.create(model_directory)

In [40]:
action_endpoint = EndpointConfig(url="http://localhost:5055/webhook")

In [41]:
agent_model_path = os.path.join(model_dir, policy_fixed_model_name)
agent = Agent.load(agent_model_path, 
                   interpreter=interpreter,
                   action_endpoint=action_endpoint)

/usr/local/lib/python3.6/dist-packages/rasa/core/policies/keras_policy.py:265: FutureWarning: 'KerasPolicy' is deprecated and will be removed in version 2.0. Use 'TEDPolicy' instead.
  current_epoch=meta["epochs"],


In [44]:
rst = await agent.parse_message_using_nlu_interpreter("count")

In [45]:
rst

{'intent': {'name': 'start_count', 'confidence': 0.9999994039535522},
 'entities': [],
 'intent_ranking': [{'name': 'start_count', 'confidence': 0.9999994039535522},
  {'name': 'bot_challenge', 'confidence': 4.4161035361867107e-07},
  {'name': 'goodbye', 'confidence': 7.065003870820874e-08},
  {'name': 'greet', 'confidence': 3.9161669462828286e-08},
  {'name': 'confirm', 'confidence': 2.158538237040375e-08}],
 'response_selector': {'default': {'response': {'name': None,
    'confidence': 0.0},
   'ranking': [],
   'full_retrieval_intent': None}},
 'text': 'count'}

In [46]:
responses = await agent.handle_text(text_message="hi", sender_id='1')

In [47]:
responses

[{'recipient_id': '1', 'text': 'Hey!'}]

In [48]:
responses = await agent.handle_text(text_message="count", sender_id='1')
responses

[{'recipient_id': '1', 'text': 'Countdown! ETA 10 🎵'}]

In [49]:
responses = await agent.handle_text(text_message="ok", sender_id='1')
responses

[{'recipient_id': '1', 'text': 'Countdown! ETA 9 🎵'}]

In [50]:
responses = await agent.handle_text(text_message="are you a bot?", sender_id='1')
responses

[{'recipient_id': '1', 'text': 'I am a bot, not a human, powered by Rasa.'},
 {'recipient_id': '1', 'text': 'I am a bot, not a human, powered by Rasa.'},
 {'recipient_id': '1', 'text': 'Countdown! ETA 7 🎵'}]

In [51]:
responses = await agent.handle_text(text_message="ok", sender_id='1')
responses

[{'recipient_id': '1', 'text': 'I am a bot, not a human, powered by Rasa.'}]

In [52]:
responses = await agent.handle_text(text_message="hi", sender_id='1')
responses

[{'recipient_id': '1', 'text': 'Hey!'}]

In [53]:
responses = await agent.handle_text(text_message="ok", sender_id='1')
responses

[{'recipient_id': '1', 'text': 'Countdown! ETA 10 🎵'}]